In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

loaded GraphQL_data/SIstakeDelegateds.csv.
loaded GraphQL_data/SIstakeLockeds.csv.
loaded GraphQL_data/SIstakeWithdrawns.csv.
loaded GraphQL_data/SIallocationCloseds.csv.
loaded GraphQL_data/SIallocationCollecteds.csv.
loaded GraphQL_data/SIstakeDepositeds.csv.


Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [14]:
delegation_events_dict = delegation_events[0]
print("EXPECTED TRUTH FROM EVENTS:")
for timestep, delegation_event in delegation_events_dict.items():
    print(f"{timestep-1}, {delegation_event[0]['delegator']}, {delegation_event[0]['shares']}")
    


EXPECTED TRUTH FROM EVENTS:
7, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
11, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55222.500000000000000000
46, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2895.065106115432857526
50, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 19300.434040769552383512
84, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3753.469591583507658440


In [16]:
print("MODELED RESULTS")
for timestep, delegation_event in delegation_events_dict.items():
    print(f"{timestep-1}, {delegation_event[0]['delegator']}, {df.iloc[timestep-1].delegators[delegation_event[0]['delegator']].shares}")
# df.iloc[8].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares

MODELED RESULTS
7, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
11, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55219.67985174618088797947460
46, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 883.1038222625919267127862494
50, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 5825.113211893787079906579872
84, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 697.1078529204305951548066311


In [19]:
print("UNITTEST RESULTS")
for timestep, delegation_event in delegation_events_dict.items():
    print(f"{timestep-1}, {delegation_event[0]['delegator']}, {delegation_event[0]['shares']}")
    print(f"{timestep-1}, {delegation_event[0]['delegator']}, {df.iloc[timestep-1].delegators[delegation_event[0]['delegator']].shares}")
    print("Equal?", delegation_event[0]['shares'] == df.iloc[timestep-1].delegators[delegation_event[0]['delegator']].shares)
# df.iloc[8].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares

UNITTEST RESULTS
7, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
7, 0x63f8168f665f79101a0e0da2313cd1391e1c5e27, 1094.500000000000000000
Equal? True
11, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55222.500000000000000000
11, 0x950d8f1e2e0ec438573a9540ca6f74913df0aeb1, 55219.67985174618088797947460
Equal? False
46, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 2895.065106115432857526
46, 0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9, 883.1038222625919267127862494
Equal? False
50, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 19300.434040769552383512
50, 0xd45d0eb21cb9dbf86012cbd574fb66d1a2c6d800, 5825.113211893787079906579872
Equal? False
84, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 3753.469591583507658440
84, 0x8e0c59ba449ac228016dee5b1bb0990a8be7f8a6, 697.1078529204305951548066311
Equal? False


In [11]:
for timestep in range(50):
    try:
        print(timestep, df.iloc[timestep].delegators['0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9'])
    except:
        pass

46 self.id='0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9', self.shares=Decimal('883.1038222625919267127862494'), self.delegated_tokens=Decimal('2985.000000000000000000')
47 self.id='0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9', self.shares=Decimal('0E-25'), self.delegated_tokens=Decimal('2069.955808415014534425592755')
48 self.id='0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9', self.shares=Decimal('0E-25'), self.delegated_tokens=Decimal('2069.955808415014534425592755')
49 self.id='0xcbb08e918e828bfd46a0e32e3365e80d7502c7d9', self.shares=Decimal('0E-25'), self.delegated_tokens=Decimal('2069.955808415014534425592755')


In [6]:
def decimal_from_value(value):
    return Decimal(value)

In [7]:
indexer_df = pd.read_csv('GraphQL_data/singleIndexer.csv')
indexer_df['epoch'] = np.floor((indexer_df['blockNumber'] - 11474307)/6500)
# scale to decimal
indexer_df['shares'] = indexer_df['shares'].apply(lambda x: decimal_from_value(x))

In [8]:
if indexer_df.iloc[1]['shares'] * Decimal(10 ** -18) == df.iloc[7].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares:
    print('Good')
else:
    print(f"{indexer_df.iloc[1]['shares'] * Decimal(10 ** -18)}, {df.iloc[7].delegators['0x63f8168f665f79101a0e0da2313cd1391e1c5e27'].shares}") 

1094.500000000000078303183128, 1094.500000000000000000
